In [1]:
!pip install pyspark
!pip install delta-spark
!pip install azure-storage-blob

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# import Necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum
from pyspark.sql.types import FloatType
from pyspark.sql.functions import concat_ws, to_timestamp, col
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql import DataFrameWriter
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import col
import os

In [24]:
# Initialize Spark session
spark = SparkSession.builder \
    .appName("UberDataPipelineAzure") \
    .getOrCreate()

In [25]:
spark

In [26]:
# Extract  this history data into a spark dataframe
df = spark.read.csv(r"raw data\ncr_ride_bookings.csv", header=True, inferSchema=True)
df.show(5)


+----------+-------------------+----------------+---------------+----------------+-------------+-------------------+-----------------+--------+--------+---------------------------+---------------------------------+-------------------------+--------------------------+----------------+-----------------------+-------------+-------------+--------------+---------------+--------------+
|      Date|               Time|      Booking ID| Booking Status|     Customer ID| Vehicle Type|    Pickup Location|    Drop Location|Avg VTAT|Avg CTAT|Cancelled Rides by Customer|Reason for cancelling by Customer|Cancelled Rides by Driver|Driver Cancellation Reason|Incomplete Rides|Incomplete Rides Reason|Booking Value|Ride Distance|Driver Ratings|Customer Rating|Payment Method|
+----------+-------------------+----------------+---------------+----------------+-------------+-------------------+-----------------+--------+--------+---------------------------+---------------------------------+--------------------

In [27]:
df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Time: timestamp (nullable = true)
 |-- Booking ID: string (nullable = true)
 |-- Booking Status: string (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Vehicle Type: string (nullable = true)
 |-- Pickup Location: string (nullable = true)
 |-- Drop Location: string (nullable = true)
 |-- Avg VTAT: string (nullable = true)
 |-- Avg CTAT: string (nullable = true)
 |-- Cancelled Rides by Customer: string (nullable = true)
 |-- Reason for cancelling by Customer: string (nullable = true)
 |-- Cancelled Rides by Driver: string (nullable = true)
 |-- Driver Cancellation Reason: string (nullable = true)
 |-- Incomplete Rides: string (nullable = true)
 |-- Incomplete Rides Reason: string (nullable = true)
 |-- Booking Value: string (nullable = true)
 |-- Ride Distance: string (nullable = true)
 |-- Driver Ratings: string (nullable = true)
 |-- Customer Rating: string (nullable = true)
 |-- Payment Method: string (nullable = true)



In [28]:
# Data cleaning and transformation
# checking for Null values
# Convert string "null" and empty strings to actual nulls
df = df.replace("null", None).replace("", None)

# Compute null counts for all columns
null_counts_row = df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).collect()[0]

for c in df.columns:
    print(f"Column: {c}, Nulls: {null_counts_row[c]}")


Column: Date, Nulls: 0
Column: Time, Nulls: 0
Column: Booking ID, Nulls: 0
Column: Booking Status, Nulls: 0
Column: Customer ID, Nulls: 0
Column: Vehicle Type, Nulls: 0
Column: Pickup Location, Nulls: 0
Column: Drop Location, Nulls: 0
Column: Avg VTAT, Nulls: 10500
Column: Avg CTAT, Nulls: 48000
Column: Cancelled Rides by Customer, Nulls: 139500
Column: Reason for cancelling by Customer, Nulls: 139500
Column: Cancelled Rides by Driver, Nulls: 123000
Column: Driver Cancellation Reason, Nulls: 123000
Column: Incomplete Rides, Nulls: 141000
Column: Incomplete Rides Reason, Nulls: 141000
Column: Booking Value, Nulls: 48000
Column: Ride Distance, Nulls: 48000
Column: Driver Ratings, Nulls: 57000
Column: Customer Rating, Nulls: 57000
Column: Payment Method, Nulls: 48000


In [29]:
df = df.select([col(c).alias(c.replace(" ", "_")) for c in df.columns])

In [30]:
df.describe().show()

+-------+----------------+--------------------+----------------+------------+---------------+-------------+------------------+------------------+---------------------------+---------------------------------+-------------------------+--------------------------+----------------+-----------------------+------------------+-----------------+------------------+------------------+--------------+
|summary|      Booking_ID|      Booking_Status|     Customer_ID|Vehicle_Type|Pickup_Location|Drop_Location|          Avg_VTAT|          Avg_CTAT|Cancelled_Rides_by_Customer|Reason_for_cancelling_by_Customer|Cancelled_Rides_by_Driver|Driver_Cancellation_Reason|Incomplete_Rides|Incomplete_Rides_Reason|     Booking_Value|    Ride_Distance|    Driver_Ratings|   Customer_Rating|Payment_Method|
+-------+----------------+--------------------+----------------+------------+---------------+-------------+------------------+------------------+---------------------------+---------------------------------+---------

In [31]:
df.show()

+----------+-------------------+----------------+--------------------+----------------+-------------+-------------------+-----------------+--------+--------+---------------------------+---------------------------------+-------------------------+--------------------------+----------------+-----------------------+-------------+-------------+--------------+---------------+--------------+
|      Date|               Time|      Booking_ID|      Booking_Status|     Customer_ID| Vehicle_Type|    Pickup_Location|    Drop_Location|Avg_VTAT|Avg_CTAT|Cancelled_Rides_by_Customer|Reason_for_cancelling_by_Customer|Cancelled_Rides_by_Driver|Driver_Cancellation_Reason|Incomplete_Rides|Incomplete_Rides_Reason|Booking_Value|Ride_Distance|Driver_Ratings|Customer_Rating|Payment_Method|
+----------+-------------------+----------------+--------------------+----------------+-------------+-------------------+-----------------+--------+--------+---------------------------+---------------------------------+-----

In [47]:
df_cleaned = df_cleaned.select([col(c).alias(c.replace(" ", "_")) for c in df_cleaned.columns])

In [32]:
# Fill up the missing values
df_cleaned = df.fillna({
    "Avg_VTAT": 0,
    "Avg_CTAT": 0,
    "Cancelled_Rides_by_Customer": 0,
    "Reason_for_cancelling_by_Customer": "NA",
    "Cancelled_Rides_by_Driver": 0,
    "Driver_Cancellation_Reason": "NA",
    "Incomplete_Rides": 0,
    "Incomplete_Rides_Reason": "NA",
    "Booking_Value": 0,
    "Ride_Distance": 0,
    "Driver_Ratings": 0,
    "Customer_Rating": 0,
    "Payment_Method": "NA"
})


In [33]:

null_counts_row = df_cleaned.select([sum(col(c).isNull().cast("int")).alias(c) for c in df_cleaned.columns]).collect()[0]
for c in df.columns:
    print(f"Column: {c}, Nulls: {null_counts_row[c]}")


Column: Date, Nulls: 0
Column: Time, Nulls: 0
Column: Booking_ID, Nulls: 0
Column: Booking_Status, Nulls: 0
Column: Customer_ID, Nulls: 0
Column: Vehicle_Type, Nulls: 0
Column: Pickup_Location, Nulls: 0
Column: Drop_Location, Nulls: 0
Column: Avg_VTAT, Nulls: 0
Column: Avg_CTAT, Nulls: 0
Column: Cancelled_Rides_by_Customer, Nulls: 0
Column: Reason_for_cancelling_by_Customer, Nulls: 0
Column: Cancelled_Rides_by_Driver, Nulls: 0
Column: Driver_Cancellation_Reason, Nulls: 0
Column: Incomplete_Rides, Nulls: 0
Column: Incomplete_Rides_Reason, Nulls: 0
Column: Booking_Value, Nulls: 0
Column: Ride_Distance, Nulls: 0
Column: Driver_Ratings, Nulls: 0
Column: Customer_Rating, Nulls: 0
Column: Payment_Method, Nulls: 0


In [34]:
# Drop both Date columns
df_cleaned = df_cleaned.drop("Date")


In [35]:
# Numeric columns
numeric_cols = ["Avg_VTAT", "Avg_CTAT", "Booking_Value", "Ride_Distance", "Driver_Ratings", "Customer_Rating"]
for c in numeric_cols:
    df_cleaned = df_cleaned.withColumn(c, col(c).cast(FloatType()))

# Flag columns
flag_cols = ["Cancelled_Rides_by_Customer", "Cancelled_Rides_by_Driver", "Incomplete_Rides"]
for c in flag_cols:
    df_cleaned = df_cleaned.withColumn(c, col(c).cast(IntegerType()))


In [36]:
df_cleaned.printSchema()

root
 |-- Time: timestamp (nullable = true)
 |-- Booking_ID: string (nullable = true)
 |-- Booking_Status: string (nullable = true)
 |-- Customer_ID: string (nullable = true)
 |-- Vehicle_Type: string (nullable = true)
 |-- Pickup_Location: string (nullable = true)
 |-- Drop_Location: string (nullable = true)
 |-- Avg_VTAT: float (nullable = true)
 |-- Avg_CTAT: float (nullable = true)
 |-- Cancelled_Rides_by_Customer: integer (nullable = true)
 |-- Reason_for_cancelling_by_Customer: string (nullable = false)
 |-- Cancelled_Rides_by_Driver: integer (nullable = true)
 |-- Driver_Cancellation_Reason: string (nullable = false)
 |-- Incomplete_Rides: integer (nullable = true)
 |-- Incomplete_Rides_Reason: string (nullable = false)
 |-- Booking_Value: float (nullable = true)
 |-- Ride_Distance: float (nullable = true)
 |-- Driver_Ratings: float (nullable = true)
 |-- Customer_Rating: float (nullable = true)
 |-- Payment_Method: string (nullable = false)



In [37]:
df_cleaned.show()

+-------------------+----------------+--------------------+----------------+-------------+-------------------+-----------------+--------+--------+---------------------------+---------------------------------+-------------------------+--------------------------+----------------+-----------------------+-------------+-------------+--------------+---------------+--------------+
|               Time|      Booking_ID|      Booking_Status|     Customer_ID| Vehicle_Type|    Pickup_Location|    Drop_Location|Avg_VTAT|Avg_CTAT|Cancelled_Rides_by_Customer|Reason_for_cancelling_by_Customer|Cancelled_Rides_by_Driver|Driver_Cancellation_Reason|Incomplete_Rides|Incomplete_Rides_Reason|Booking_Value|Ride_Distance|Driver_Ratings|Customer_Rating|Payment_Method|
+-------------------+----------------+--------------------+----------------+-------------+-------------------+-----------------+--------+--------+---------------------------+---------------------------------+-------------------------+------------

In [38]:
# Creating tables
customers = df_cleaned.select("Customer_ID", "Customer_Rating")\
                      .dropDuplicates()\
                      .withColumn("Customer_ID", monotonically_increasing_id())\
                      .select("Customer_ID", "Customer_ID", "Customer_Rating")

customers.show(5, truncate=False)

+-----------+-----------+---------------+
|Customer_ID|Customer_ID|Customer_Rating|
+-----------+-----------+---------------+
|0          |0          |4.1            |
|1          |1          |0.0            |
|2          |2          |0.0            |
|3          |3          |4.7            |
|4          |4          |4.3            |
+-----------+-----------+---------------+
only showing top 5 rows


In [39]:

# Driver table
drivers = df_cleaned.select("Driver_Ratings")\
                    .dropDuplicates()\
                    .withColumn("Driver_ID", monotonically_increasing_id())\
                    .select("Driver_ID", "Driver_Ratings")

drivers.show(5, truncate=False)

+---------+--------------+
|Driver_ID|Driver_Ratings|
+---------+--------------+
|0        |5.0           |
|1        |3.8           |
|2        |4.6           |
|3        |3.4           |
|4        |4.2           |
+---------+--------------+
only showing top 5 rows


In [40]:
# Combine pickup and drop locations
pickup = df_cleaned.select("Pickup_Location").withColumnRenamed("Pickup_Location", "Location_Name")
drop = df_cleaned.select("Drop_Location").withColumnRenamed("Drop_Location", "Location_Name")

all_locations = pickup.union(drop).dropDuplicates()

# Add a unique Location_ID
locations = all_locations.withColumn("Location_ID", monotonically_increasing_id())\
                         .select("Location_ID", "Location_Name")

locations.show(5, truncate=False)

+-----------+----------------+
|Location_ID|Location_Name   |
+-----------+----------------+
|0          |Huda City Centre|
|1          |Mayur Vihar     |
|2          |Noida Sector 62 |
|3          |Qutub Minar     |
|4          |Ardee City      |
+-----------+----------------+
only showing top 5 rows


In [43]:
# Rename columns to remove spaces
df_cleaned = df_cleaned.select([col(c).alias(c.replace(" ", "_")) for c in df_cleaned.columns])
drivers = drivers.select([col(c).alias(c.replace(" ", "_")) for c in drivers.columns])
locations = locations.select([col(c).alias(c.replace(" ", "_")) for c in locations.columns])

# Map Driver_Ratings to Driver_ID
bookings = df_cleaned.join(drivers, on="Driver_Ratings", how="left")

# Map Pickup Location to Location_ID
bookings = bookings.join(
    locations.withColumnRenamed("Location_ID", "Pickup_Location_ID"),
    bookings["Pickup_Location"] == locations["Location_Name"],
    how="left"
).drop("Location_Name")

# Map Drop Location to Location_ID
bookings = bookings.join(
    locations.withColumnRenamed("Location_ID", "Drop_Location_ID"),
    bookings["Drop_Location"] == locations["Location_Name"],
    how="left"
).drop("Location_Name")

# Select final columns in desired order
bookings = bookings.select(
    "Booking_ID", "Customer_ID", "Driver_ID", "Booking_Status", "Vehicle_Type",
    "Pickup_Location_ID", "Drop_Location_ID", "Avg_VTAT", "Avg_CTAT",
    "Cancelled_Rides_by_Customer", "Reason_for_cancelling_by_Customer",
    "Cancelled_Rides_by_Driver", "Driver_Cancellation_Reason",
    "Incomplete_Rides", "Incomplete_Rides_Reason", "Booking_Value",
    "Ride_Distance", "Payment_Method"
)

bookings.show(5, truncate=False)


+----------------+----------------+---------+---------------+-------------+------------------+----------------+--------+--------+---------------------------+---------------------------------+-------------------------+--------------------------+----------------+-----------------------+-------------+-------------+--------------+
|Booking_ID      |Customer_ID     |Driver_ID|Booking_Status |Vehicle_Type |Pickup_Location_ID|Drop_Location_ID|Avg_VTAT|Avg_CTAT|Cancelled_Rides_by_Customer|Reason_for_cancelling_by_Customer|Cancelled_Rides_by_Driver|Driver_Cancellation_Reason|Incomplete_Rides|Incomplete_Rides_Reason|Booking_Value|Ride_Distance|Payment_Method|
+----------------+----------------+---------+---------------+-------------+------------------+----------------+--------+--------+---------------------------+---------------------------------+-------------------------+--------------------------+----------------+-----------------------+-------------+-------------+--------------+
|"""CNR588430